<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/mmlu_pro/(41)pro%EA%B3%A0%EC%B9%9C%EA%B1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain-upstage langchain-community pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: langchain-core


In [2]:
pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=ab3ea66d44e15d1fc65958a42019c5f621e6054180fdf27dd64a89743d9aab0a
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


In [3]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 65.4 MB/s eta 0:00:00


In [7]:
import os
import json
from typing import List, Tuple

from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

import wikipediaapi
import pandas as pd

In [8]:
cd /content/drive/MyDrive/Colab Notebooks/2025_2/nlp

/content/drive/MyDrive/Colab Notebooks/2025_2/nlp


In [22]:
df = pd.read_csv("testset.csv")

In [23]:
UPSTAGE_API_KEY = "up_g7T2cQoLKZH6Oi2n4MHOW706XAdSs"

In [24]:
llm_classifier = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.1, #거의 항상 같은 결과, 안정적이게
)

In [25]:
llm_solver = ChatUpstage(
    api_key=UPSTAGE_API_KEY,
    model="solar-pro2",
    temperature=0.2, #좀 더 유연한 답변을 내도록
)

In [26]:
category_index = {
    "law":        "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/mmlu_category/law",
    "psychology": "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/mmlu_category/psychology",
    "business":   "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/mmlu_category/business",
    "philosophy": "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/mmlu_category/philosophy",
    "history":    "/content/drive/MyDrive/Colab Notebooks/2025_2/nlp/mmlu_category/history",
}
categories = list(category_index.keys())

In [27]:
emb = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large")

# 카테고리별 faiss 한번에 로드해서 캐시
vectorstore = {}
for cat, path in category_index.items():
    vectorstore[cat] = FAISS.load_local(
        folder_path=path,
        embeddings=emb,
        allow_dangerous_deserialization=True,
    )

wiki = wikipediaapi.Wikipedia(user_agent= "NLP-RAG/1.0", language = 'en')

**1st LLM**:

- return category

- return 3 keywords

In [28]:
category_prompt_template = """
You are an expert exam classifier for MMLU-Pro

There are 5 possible categories:
- law
- psychology
- business
- philosophy
- history

[Task]
Given a multiple-choice exam question (including all options),
1. Choose one best category from the list above.
2. Extract exactly 3 important keywords (one noun that consists of single word or short phrase)
   that will be useful to search textbooks and Wikipedia.
   Keywords should be as specific and accurate as possible (e.g., "consent",
   "cognitive dissonance", "Keynesian economics").

[Output format]
Return a valid JSON with the following fields:
- "category": one of ["law","psychology","business","philosophy","history"]
- "keywords": a list of exactly 3 strings

This is an output example:
{{
  "category": "psychology",
  "keywords": ["informed consent", "assent", "child counseling"]
}}

[Question]
{question}
""".strip()

category_prompt = ChatPromptTemplate.from_template(category_prompt_template)

In [36]:
# def classify_and_extract_keywords(question_text: str):
#     messages = category_prompt.format_messages(question=question_text)
#     #메시지를 upstage api로 보내서 llm의 응답을 resp에 넣
#     resp = llm_classifier.invoke(messages)
#     raw = resp.content.strip()

#     data = json.loads(raw)

#     category = data["category"].strip().lower()
#     keywords = [k.strip() for k in data.get("keywords", [])][:3]

#     return category, keywords

In [37]:
def classify_and_extract_keywords(question_text: str):
    messages = category_prompt.format_messages(question=question_text)
    resp = llm_classifier.invoke(messages)
    raw = resp.content.strip()

    # 🔍 디버깅용: 처음엔 한두 번 찍어서 어떤 출력이 나오는지 확인해봐
    # print("RAW CATEGORY OUTPUT:", repr(raw))

    if not raw:
        # 완전 빈 응답이면 기본값으로 대충 처리 (죽지 않게)
        print("[WARN] Empty LLM output for category, fallback to 'history'")
        return "history", ["history", "war", "europe"]

    # 1차 시도: 전체를 JSON으로 해석
    try:
        data = json.loads(raw)
    except json.JSONDecodeError:
        # 2차 시도: 문자열 안에서 {...} 구간만 잘라서 해석
        start = raw.find("{")
        end = raw.rfind("}")
        if start == -1 or end == -1:
            print("[WARN] No JSON object found in output, fallback to 'history'")
            return "history", ["history", "war", "europe"]

        json_str = raw[start:end+1]
        try:
            data = json.loads(json_str)
        except json.JSONDecodeError as e:
            print("[WARN] JSON parse failed again:", e)
            return "history", ["history", "war", "europe"]

    category = data.get("category", "history").strip().lower()
    keywords = data.get("keywords", [])
    keywords = [str(k).strip() for k in keywords][:3]

    if not keywords:
        keywords = ["history", "war", "europe"]

    return category, keywords

In [38]:
def fetch_wiki_context(keywords, max_chars_per_page=1200):
    snippets = []
    for kw in keywords:
        try:
            page = wiki.page(kw)
            if not page.exists():
                print(f"[WARN] page for '{kw}' does not exist")
                continue
            text = page.text[:max_chars_per_page]
            snippets.append(f"[Wikipedia: {page.title}]\n{text}")
        except Exception as e:
            print(f"[WARN] wikipediaapi fetch failed for '{kw}': {e}")
            continue

    return "\n\n".join(snippets)

In [39]:
TOP_K_TEXTBOOK = 5

def build_context_for_solver(question_text: str, category: str, keywords):
    # 1) 카테고리별 textbook RAG
    vs = vectorstore[category]
    docs = vs.similarity_search(question_text, k=TOP_K_TEXTBOOK)
    textbook_context = "\n\n".join(
        f"[Textbook Doc {i+1}] {d.page_content}" for i, d in enumerate(docs)
    )

    # 2) Wikipedia context
    wiki_context = fetch_wiki_context(keywords)

    full_context = f"""\
=== TEXTBOOK CONTEXT ({category}) ===
{textbook_context}

=== WIKIPEDIA CONTEXT (keywords: {', '.join(keywords)}) ===
{wiki_context}
"""
    return full_context

**2nd LLM**:

- return CoT
- return 최종 정답

In [40]:
SOLVER_PROMPT_TMPL = """
You are an expert exam solver.

Use the following context (textbook + Wikipedia) to answer the
multiple-choice question. Carefully reason step by step, comparing each option
with the evidence.

At the end, output the final answer in the format:

Final Answer: X

where X is a single capital letter (A, B, C, D, ...).

If the context is incomplete, choose the MOST reasonable answer but do not
contradict the given evidence.

[CONTEXT]
{context}

[QUESTION]
{question}

Now reason step by step, then give the final answer.
""".strip()

solver_prompt = ChatPromptTemplate.from_template(SOLVER_PROMPT_TMPL)


In [41]:
def solve_mmlu_with_rag_and_wiki(question_text: str):
    # 1) 1단계 LLM: 카테고리 + 키워드
    category, keywords = classify_and_extract_keywords(question_text)
    if category not in vectorstore:
        print(f"[WARN] Unknown category '{category}', fallback to 'business'")
        category = "business"

    # 2) 컨텍스트 구성 (FAISS + Wikipedia)
    context = build_context_for_solver(question_text, category, keywords)

    # 3) 2단계 LLM: CoT로 정답 도출
    messages = solver_prompt.format_messages(
        context=context,
        question=question_text,
    )
    resp = llm_solver.invoke(messages)
    raw = resp.content.strip()

    # 4) "Final Answer: X"에서 X 추출
    final_letter = None
    for line in raw.splitlines()[::-1]:  # 아래쪽부터 검색
        line = line.strip()
        if line.upper().startswith("FINAL ANSWER"):
            # 예: "Final Answer: D"
            parts = line.split(":")
            if len(parts) >= 2:
                cand = parts[1].strip()
                if cand and "A" <= cand[0] <= "Z":
                    final_letter = cand[0]
                    break

    # 그래도 못 찾으면, 전체 텍스트에서 대문자 한 글자 검색
    if final_letter is None:
        for ch in raw:
            if "A" <= ch <= "Z":
                final_letter = ch
                break

    return {
        "category": category,
        "keywords": keywords,
        "raw_reasoning": raw,       # 디버깅/분석용
        "final_answer": final_letter,
    }

**testset 실행 예시**:
- baseline.csv

In [42]:
import pandas as pd

df = pd.read_csv("testset.csv")[26:]  # 질문, 선택지 다 합쳐진 컬럼 가정
QUESTION_COL = "prompts"

results = []
for i, row in df.iterrows():
    q = row[QUESTION_COL]
    print(f"[{i}] Solving: {q[:60]}...")
    out = solve_mmlu_with_rag_and_wiki(q)
    results.append(out["final_answer"])
    df.loc[i, "pred_category"] = out["category"]
    df.loc[i, "kw1"] = out["keywords"][0] if len(out["keywords"]) > 0 else ""
    df.loc[i, "kw2"] = out["keywords"][1] if len(out["keywords"]) > 1 else ""
    df.loc[i, "kw3"] = out["keywords"][2] if len(out["keywords"]) > 2 else ""
    df.loc[i, "rag_cot_answer"] = out["final_answer"]
    df.loc[i, "cot_full"] = out["raw_reasoning"]

df.to_csv("baseline-pro.csv", index=False)
print("저장 완료:baseline-pro.csv")


[26] Solving: QUESTION27) A man is at home in his apartment, alone, late a...
[27] Solving: QUESTION28) What do Homo sapiens and Australopithecus afaren...
[28] Solving: QUESTION29)This question refers to the following information...
[WARN] page for 'Tang relations' does not exist
[WARN] page for 'frontier peoples' does not exist
[29] Solving: QUESTION30)Homo erectus differed from Homo habilis in which ...
[30] Solving: QUESTION31)During the manic phase of a bipolar disorder, ind...
[WARN] page for 'manic phase' does not exist
[WARN] page for 'high self-esteem' does not exist
[31] Solving: QUESTION32) This question refers to the following informatio...
[32] Solving: QUESTION33) You receive a phone call from Hermann H., age 28...
[WARN] page for 'ethical psychologist' does not exist
[33] Solving: QUESTION34) During the second stage of Kohlberg’s preconvent...
[WARN] page for 'Kohlberg’s theory' does not exist
[WARN] page for 'preconventional level' does not exist
[34] Solving: QUESTION3

**정확도**

In [46]:

import pandas as pd

# 1. 파일 경로
GT_PATH = "testset.csv"          # 정답 파일
PRED_PATH = "baseline-pro.csv"       # 출력 파일

# 2. 컬럼 이름
GT_COL = "answers"               # testset.csv에서 정답 컬럼
PRED_COL = "rag_cot_answer"      # baseline.csv에서 예측 컬럼

# 3. csv 로드
gt_df = pd.read_csv(GT_PATH)[26:]      # 26번 문제부터라고 가정
pred_df = pd.read_csv(PRED_PATH)

# 4. Series 추출
gt = gt_df[GT_COL].astype(str)
pred = pred_df[PRED_COL].astype(str)

# 5. 정규화 함수
def normalize_choice(x: str) -> str:
    x = x.strip().upper()
    for ch in x:
        if "A" <= ch <= "Z":
            return ch
    return x

# 인덱스 리셋이 포인트
gt_norm = gt.apply(normalize_choice).reset_index(drop=True)
pred_norm = pred.apply(normalize_choice).reset_index(drop=True)

print(len(gt_norm), len(pred_norm))  # 둘 다 25 나오는지 체크 한번 해보고

# 6. 정확도 계산
correct = (gt_norm == pred_norm)
accuracy = correct.mean()

print(f"총 문제 수: {len(gt_norm)}")
print(f"정답 개수: {correct.sum()}")
print(f"정확도: {accuracy * 100:.2f}%")

24 24
총 문제 수: 24
정답 개수: 10
정확도: 41.67%
